In [7]:
%pip install pandas
%pip install openpyxl
%pip install ipywidgets

In [1]:
from BasicTerm_S_nomx import BasicTerm_S

In [2]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [3]:
interact(lambda p: BasicTerm_S.Projection[p].result_pv(), p=(1,1000))

interactive(children=(IntSlider(value=500, description='p', max=1000, min=1), Output()), _dom_classes=('widget…

<function __main__.<lambda>(p)>

In [4]:
interact(lambda p: BasicTerm_S.Projection[p].result_cf(), p=(1,1000))

interactive(children=(IntSlider(value=500, description='p', max=1000, min=1), Output()), _dom_classes=('widget…

<function __main__.<lambda>(p)>